In [1]:
import numpy as np
import bentoml
from bentoml.io import NumpyNdarray
from bentoml.io import JSON
from annoy import AnnoyIndex


import transformers
transformers.logging.set_verbosity_error()
from torch.utils.data import DataLoader 

from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import TFDistilBertModel, DistilBertTokenizerFast
import torch
from scipy.sparse import csr_matrix
from tqdm import tqdm

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
import pickle5 as pickle
import sqlite3

nltk.download('omw-1.4')


[nltk_data] Downloading package omw-1.4 to /home/rkozik/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
import pickle5 as pickle
import sqlite3


# Interface lemma tokenizer from nltk with sklearn
class LemmaTokenizer:
    ignore_tokens = [',', '.', ';', ':', '"', '``', "''", '`']
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc) if t not in self.ignore_tokens]


In [145]:

class MyCustomUnpickler(pickle.Unpickler):
    def find_class(self, module, name):
        return super().find_class(module, name)

print("load vectorizer")    
with open(f'../../pickles/swarog_data/tfidf_vectorizer_full.pickle', 'rb') as handle:
    unp = MyCustomUnpickler(handle)
    tfidf_vectorizer = unp.load()

load vectorizer


In [217]:
_sentence = """Last week, I had what I've been calling a "pothole moment"-- a flare of intense feeling that tripped me up. At our fall conference, my son's middle school guidance counselor suggested that the pandemic years would have been the ideal time for him to learn to type, even though, to my knowledge, during those years, his school never once mentioned typing. She told me that no, the school isn't going to circle back to teach it, though it's clearly increasingly necessary. But maybe, the counselor suggested brightly, I could just teach my son now?  As it turned out, the call to "just" do anything made my breath come short. I mean, didn't I just nearly break my own soul keeping my two kids from loneliness and despair for two tough years, making sure that they learned a few things for when the world woke up again? Didn't I teach swimming when there were no swim lessons, and every last game I knew during the long summer when there was no camp? Isn't it a miracle that either kid knows any math at all?  In front of the guidance counselor, I found myself growing tense. "No," I said. "I cannot 'just' teach anything right now." I heard the little snarl in my throat. Heart racing, I stepped back to find a new tone. In perhaps an overshare, or perhaps just what the moment needed, I told the woman in front of me that I'd just been swamped by sudden deep grief -- not about typing, but for all that's been broken, for the crazy years we've been through.  And, with a bit of grace, the counselor saw me. She was kind. She got it. "It's normal," she said. "We're all sort of that way."  So that's me, waving my hand, sharing my pothole. I think I'm not alone, that a lot of our roads these days can veer into the uneven without much warning. As we move towards a holiday season where many of us will gather with family and friends, it's interesting to notice all we're holding: The lingering pandemic and its aftereffects. The long shadow of a world at war. Grief about climate change. Grief about gun violence. Fear about the price of gas and groceries. And other rifts -- in our families, in our towns, in our country. Some of these feel like they will never be mended.  Yet alongside noting the general tenderness I find in myself and others, I've also been thinking about joy, what it is, how to find it. As it happens, I'm not alone. There are at least three books this fall that take up the call to think about joy and ask us to think with them. As the holidays approach, I've been reading them. I don't know about you, but sometimes the call to holiday joy puts me in an uneasy place. But these three books are actually about moving past what feels forced and fake towards what might feel genuine and alive, together.  In short, as we all begin to try to think about what it might mean to really be near one another, these books have a lot to offer now.  The books -- and this is not the start of a joke -- are actually by a poet, a psychoanalyst and a pastor, three lovely people who certainly should all walk into a bar together to chat. Poet Ross Gay's "Inciting Joy" makes the pursuit of joy feel positively radical. Poet and psychoanalyst Nuar Alsadir's "Animal Joy" is, among other things, an exploration of the unexpected art of really laughing. And in "How to Begin When Your World Is Ending, A Spiritual Field Guide to Joy Despite Everything," pastor Molly Phinney Baskette -- who's lived through cancer and tended the griefs of her congregants -- doesn't quite offer a how-to manual, but in some ways, it comes close.  These three books share common wisdoms: That joy is not really about scripted, expected or store-bought things. It is more about the unscripted realm of deep connection, the willingness to be vulnerable, to be present with sorrow, to be honest, to be surprised. It's sharing the unexpected, rather than holding it back. It's also about the moments when, despite all, we feel met.  Tellingly, none of these books are about joy alone. There's an awful lot of suffering baked in -- the loss of parents, the loss of children, the loss of the selves we thought we were. Even what is funny can be upending: "Every joke is a tiny revolution" says Alsadir, quoting George Orwell, also noting that many standup routines could be tragedy in another tone.  As a poet and psychoanalyst, Alsadir is fascinated by the way moments that make us laugh and feel deeply happen when the tender and taboo parts of life see the light. When something -- our deep laughter, our authentic grief -- upends the narratives we'd rather keep steady, we also discover the possibility for transformation.  For Phinney Baskette, the pastor, the process of joy comes in trusting that these broken open moments -- where we may feel most lost -- are actually the places where we might just connect more deeply to ourselves, to others, and to the strength we need to go on. Joy emerges when we can call out to people -- as she did when she was presiding over Holy Week after discovering she had cancer and found herself unexpectedly asking three congregants to pray for her, instead.  Perhaps my favorite description of joy comes from Gay's book. In what's a very long metaphor, a shaggy dog parable of its own, Gay describes joy as the potluck we come to, to greet our old companion, Sorrow. The potluck is vast: "We put our mechanic on the list, our chiropractor, and the neighbors we wave at but not much more than that."  And then it turns out that as we bring our dishes, and as we put out our index cards in front of them, saying whether they have nuts or gluten or dairy, as we greet each other, introducing our sorrows, something uproarious happens: The party gets joyous, and zany, and most importantly, real. Suddenly we are "sweaty, stomping and shaking, tearing it up, the pictures falling off the walls, the books from the shelves, some logs ablaze even spilling from the stove, riotous this care, this carrying this excitement, this joy." Suddenly, in other words, having invited in our sorrows, we can finally dance at the party of our lives.  As we come to the Thanksgiving holiday, I can't help but think about this, about bringing our dishes to friends and loved ones, and bringing also what is real, hungry, and vulnerable too.  I had a glimpse of this lately, how good it could be. I was recently away for a weekend among some brilliant writer friends, some of the smartest women I know. Many of us haven't seen each other in years. We started in delight just to be together again. We shared new work, wine, ideas, hugs.  But later in the weekend, something deeper started to happen. We brought more of ourselves to the table. I heard from a friend with an eating disorder, one who had cancer during the pandemic, one losing her ex-husband to drug addiction, one who'd been shut out of a dream job, and is trying -- quite bravely -- to work around bitterness. I was able to talk about my potholes, my exhaustion, my long burnout and the feeling trying to chart my way in the dark.  We all laughed a lot. I cried in front of my friend Nicole. "It's totally ok to be fragile," she said. "We're all fragile now."  Yeah, and maybe we're ready to be joyful, too. It's winter. It's cold. The holidays are coming. We're about to try to find light in darkness. We're about to gather with a bunch of people we love, and those we try hard to.  Sometimes those people we try to love are the very same people who also make us feel, and sometimes the feelings are challenging. In other words, the holidays are full of potholes. The holidays are coming. The world is going to try to sell us the kind of joy we can buy in stores. What I'm hoping for also -- for each of us -- is that we find some togetherness, some real community. I'm hoping that we make it to that potluck, where we laugh and laugh and laugh."""
vec = tfidf_vectorizer.transform([_sentence]).toarray()[0]

In [218]:
np.where(vec > 0)[0]

array([  4799,   5318,  10634, 104568, 108471, 108476, 112439, 113342,
       116824, 125742, 127732, 127815, 128276, 131303, 133993, 134985,
       138461, 140364, 143631, 144275, 145980, 146119, 150779, 152520,
       152696, 154983, 156977, 159340, 161245, 164801, 173827, 174212,
       179546, 183414, 183638, 183858, 185022, 185081, 185155, 185179,
       185906, 186387, 188989, 189807, 191053, 192435, 193948, 194163,
       194836, 195740, 197763, 197995, 199252, 204161, 205273, 205799,
       207095, 207259, 209696, 210853, 213886, 216301, 216792, 217653,
       220920, 222465, 222842, 223659, 223980, 224267, 226588, 227271,
       227631, 227671, 234066, 234717, 237132, 237940, 245405, 246235,
       247139, 247211, 248661, 252394, 252523, 252556, 254180, 257508,
       257514, 258159, 258258, 264711, 264735, 265253, 265621, 268521,
       272574, 273709, 278952, 282587, 286154, 287912, 295664, 296367,
       296475, 297443, 298458, 298940, 299492, 300072, 303698, 303994,
      

In [219]:
import re

_names = tfidf_vectorizer.get_feature_names_out()
_chain = sorted(list(zip(vec[np.where(vec > 0)], _names[np.where(vec > 0)[0]])), key=lambda tup: -tup[0])[0:]
" ".join([re.sub(r'[^a-zA-Z0-9]', ' ', t[1]) for t in _chain])

'joy    holiday pothole grief book potluck psychoanalyst sorrow feel poet counselor baskette phinney laugh teach moment pastor try  m friend congregant u typing pandemic gay find front come greet cancer three think feeling deep fragile sometimes lot alone loss dish perhaps also unexpected call long overshare gather guidance tone together aftereffect uproarious joke suddenly world vulnerable actually real upends noting school hoping sharing snarl broken way mended deeply riotous how to store bought people zany standup shaggy tenderness something gluten year chiropractor bringing really make togetherness tellingly happens parable   unscripted burnout one veer put stomping maybe towards sweaty upending suggested weekend tripped  just joyous kid holding brightly swamped joyful thing loneliness quite begin happen light uneven smartest bitterness stove exhaustion son fall heard orwell spilling ablaze trusting fascinated short know bravely ex husband scripted molly offer flare metaphor taboo 

In [199]:
_chainstr

'foxconn worker zhengzhou yuan cnn bonus livestreams protest dormitory recruit'

In [220]:
for _range in range(0,10):
    _names = tfidf_vectorizer.get_feature_names_out()
    _chain = sorted(list(zip(vec[np.where(vec > 0)], _names[np.where(vec > 0)[0]])), key=lambda tup: -tup[0])[:10-_range]
    _chainstr = " ".join([re.sub(r'[^a-zA-Z0-9]', '', t[1]) for t in _chain])

    conn = sqlite3.connect('../../pickles/swarog_data/swarog.sqlite')
    c = conn.cursor()
    c.execute(f"""select rowid,label, dataset, body from rawsearch where body match '{_chainstr}'""")
    r=c.fetchall()
    print(10-_range,"->",len(r), " ", _chainstr)

10 -> 0      joy  holiday pothole grief book potluck psychoanalyst sorrow feel
9 -> 0      joy  holiday pothole grief book potluck psychoanalyst sorrow
8 -> 0      joy  holiday pothole grief book potluck psychoanalyst
7 -> 0      joy  holiday pothole grief book potluck
6 -> 0      joy  holiday pothole grief book
5 -> 0      joy  holiday pothole grief
4 -> 0      joy  holiday pothole
3 -> 100      joy  holiday
2 -> 1526      joy 
1 -> 1526      joy


In [ ]:
select rowid from rawsearch where body match 'cough' limit 10;

In [260]:
_names[0:11]

array(['\x10the', '\x10there', '!', '#', '$', '%', '&', "'+currentpage",
       "'-", "'-et", "'-ratgeber"], dtype=object)

In [328]:
_sentence = """5G technology spreads covid 19"""
vec = tfidf_vectorizer.transform([_sentence]).toarray()[0]

_names = tfidf_vectorizer.get_feature_names_out()
_words = sorted(list(zip(vec[np.where(vec > 0)], _names[np.where(vec > 0)[0]])), key=lambda tup: -tup[0])

conn = sqlite3.connect('../../gui/swarog/pickles/swarog.sqlite')
_resp=[]
_wordhits = {}
for _range in range(0,min(20,len(_words))):
    x=_words[_range]
    _chain = (x[0],re.sub(r'[^a-zA-Z0-9]', ' ', x[1]).split(" ")[0])
    
    if len(_chain[1]) == 0:
        continue

    print(_chain)
    
    c = conn.cursor()
    c.execute(f"""select rowid,_chain[1] as word from rawsearch where body match '{_chain[1]}' """)
    docsids = c.fetchall()
    r=[x[0] for x in docsids]
    for x in docsids:
        if not x[0] in _wordhits:
            _wordhits[x[0]] = []
        _wordhits[x[0]].append(_x[1])
    _resp.extend(r)
    
    
from collections import Counter
ctr=Counter(_resp)
common = ctr.most_common(10)

print("common",common)
# ids = [str(x[0]) for x in common]

# c = conn.cursor()
# c.execute(f"""select label, dataset, rowid from raw where rowid IN ({",".join(ids)})""")
# #print( c.fetchall())

# #_resp

resp = []
for _docid in common:
    c.execute(f"""select label, dataset, body from rawsearch where rowid = {_docid[0]} """)
    _results_hits = c.fetchall()[0]
    resp.append({
        'text':_results_hits[2],
        'label':_results_hits[0],
        'dataset':_results_hits[1],
        'distance':1.0 - _docid[1]*1.0/min(20,len(_words))})
print(resp)

conn.close()

(0.5870630363804262, '5g')
(0.5666778204715793, 'covid')
(0.3391582943275229, '19')
(0.3355310771421048, 'spread')
(0.32653297959330074, 'technology')
common [(10120, 5), (10338, 5), (10532, 5), (10538, 5), (11931, 5), (11949, 5), (12384, 5), (12392, 5), (17320, 5), (19788, 5)]
[{'text': 'The Statement A Facebook video claiming to show a piece of 5G equipment labelled “COV.19” is being shared on Facebook as conspiracy theorists and 5G opponents target telecom masts in the UK and other parts of Europe. The May 14 Facebook video features a man wearing a face mask, a clean yellow hardhat and and orange hi-vis jacket, who claims to have been “working erecting 5G masts on towers like the one behind me … while everybody else has been in lockdown”. The man claims he has opened a “5G kit” despite being “explicitly asked not to”. The camera then turns to a piece of circuit board he is holding. In the upper left corner of the board is a silver component with a label on it that reads “COV.19”. Th

In [290]:
sorted(wordmap.items(), key=lambda tup: -tup[1])

[(10308, 1.7199365236796214),
 (12148, 1.7199365236796214),
 (12155, 1.7199365236796214),
 (12174, 1.7199365236796214),
 (12178, 1.7199365236796214),
 (12180, 1.7199365236796214),
 (138049, 1.6056183222933023),
 (166307, 1.6056183222933023),
 (172402, 1.6056183222933023),
 (179201, 1.6056183222933023),
 (5406, 1.4950357319228518),
 (11815, 1.4285678469191625),
 (12012, 1.397147251214422),
 (12118, 1.397147251214422),
 (12176, 1.397147251214422),
 (12177, 1.397147251214422),
 (12185, 1.397147251214422),
 (12326, 1.397147251214422),
 (12422, 1.397147251214422),
 (18553, 1.3785328913457005),
 (23526, 1.3785328913457005),
 (23811, 1.3785328913457005),
 (10387, 1.34711229564096),
 (11336, 1.34711229564096),
 (12043, 1.34711229564096),
 (12154, 1.34711229564096),
 (12195, 1.34711229564096),
 (12387, 1.34711229564096),
 (12795, 1.34711229564096),
 (23453, 1.34711229564096),
 (26033, 1.34711229564096),
 (17024, 1.282829049828103),
 (19212, 1.282829049828103),
 (36254, 1.282829049828103),
 (126

[(19311, 6),
 (12039, 5),
 (10308, 5),
 (16952, 5),
 (16740, 5),
 (33780, 5),
 (12013, 5),
 (17736, 5),
 (16864, 5),
 (10297, 5)]

In [156]:
tfidf_vectorizer.get_feature_names_out()[np.where(vec > 0)]

array(['breath', 'cough', 'covid-19', 'dry', 'hallmark', 'shortness',
       'sign'], dtype=object)

In [138]:

def get_related_articles(txt):
    vec = tfidf_vectorizer.transform([txt]).toarray()[0]
    vec[np.where(vec > 0)]=1
    hits = tfidf.get_nns_by_vector(vec, 10, search_k=10000000000, include_distances=True)
    print("-->",hits)
    resp = []
    conn = sqlite3.connect('../../pickles/swarog_data/swarog.sqlite')
    c = conn.cursor()
    for index,_id in enumerate(hits[0]):
        c.execute("""select raw.body,raw.label, raw.dataset 
                from raw join tfidf on (tfidf.gid=raw.rowid) 
                where tfidf.rowid=?""",[_id+1])
        r=c.fetchall()
        resp.append({
            'text':r[0][0],
            'label':r[0][1],
            'dataset':r[0][2],
            'distance':hits[1][index]})
    conn.close()
    return resp



tfidf = AnnoyIndex(7000, 'hamming')
print("load tfidf annoy index...")
tfidf.load('../../pickles/swarog_data/swarog_tfidf.ann')


hits = tfidf.get_nns_by_vector(vec, 10, search_k=1000000, include_distances=True)
hits

load tfidf annoy index...


([732, 3465, 4416, 5744, 8478, 34, 172, 201, 423, 802],
 [4.0, 4.0, 4.0, 4.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0])

In [128]:
get_related_articles("dry cough, and shortness of breath are hallmark signs COVID-19")

--> ([732, 4416, 5744, 8478, 172, 1978, 2071, 3130, 3377, 3941], [4.0, 4.0, 4.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0])


[{'text': 'COVID-19 is mutating and is now more contagious than before.',
  'label': 1,
  'dataset': 'covid_fake_news',
  'distance': 4.0},
 {'text': 'COVID-19 is transmitted by farting',
  'label': 1,
  'dataset': 'covid_fake_news',
  'distance': 4.0},
 {'text': 'Eucalyptus prevents or eliminates COVID-19.',
  'label': 1,
  'dataset': 'covid_fake_news',
  'distance': 4.0},
 {'text': 'If you have a runny nose or cough, you donâ€™t have COVID-19.',
  'label': 1,
  'dataset': 'covid_fake_news',
  'distance': 4.0},
 {'text': 'Authorities discreetly re-authorised hydroxychloroquine for COVID-19',
  'label': 1,
  'dataset': 'covid_fake_news',
  'distance': 5.0},
 {'text': 'Aspirin cures COVID-19.',
  'label': 1,
  'dataset': 'covid_fake_news',
  'distance': 5.0},
 {'text': 'Hydroxychloroquine cures COVID-19.',
  'label': 1,
  'dataset': 'covid_fake_news',
  'distance': 5.0},
 {'text': 'Ozone serves as a disinfectant against COVID-19.',
  'label': 1,
  'dataset': 'covid_fake_news',
  'distan